In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

### 论文中的数据分析 
Clinical Outcome and Biological Predictors of Relapse After Nephrectomy Only for Very Low-risk Wilms Tumor

极低风险肾母细胞瘤肾切除术后的临床结果和复发的生物学预测因素

链接: https://pmc.ncbi.nlm.nih.gov/articles/PMC5145762/

In [26]:
dt =pd.read_csv("wt_relapsed.csv", sep="\t")
dt

,UPN,Age at Diagnosis (Months),Tumor Weight at Diagnosis (Grams),Histology at Diagnosis,ILN at Diagnosis,PLN at Diagnosis,Time to Relapse (Days)*,Site of First Relapse,Symptoms at Relapse,Chemotherapy,Radiotherapy
0,1,21,250,Mixed,NaN,NaN,68,Lung,No,DD4A,Whole lung
1,78,9,261,Blastemal,NaN,NaN,138,Lungs,No,DD4A,Whole lung
2,154,5,508,Blastemal,NaN,NaN,376,Retroperitoneum,No,DD4A,Flank
3,165,14,505,Blastemal,NaN,NaN,120,Lung,Yes,DD4A,Whole lung
4,399,10,271,Mixed,NaN,NaN,91,Retroperitoneum,No,DD4A,Flank
5,434,3,254,Blastemal,NaN,NaN,94,Right renal fossa,No,DD4A,Delayed Flank
6,450,15,178,Epithelial tubular,NaN,NaN,80,Contra lateral kidney,No,DD4A,Flank
7,476,12,252,Mixed,NaN,NaN,186,Right renal fossa/Lungs,No,DD4A,Flank/Whole lung
8,500,20,490,Mixed,Present-multiple,NaN,178,Abdomen,Yes,DD4A,Whole abdomen
9,508,5,270,Mixed,Present,NaN,1305,Lung,No,DD4A,No


In [25]:
from typing import List

def get_origin_data(x:List[int|float]):
    return ",".join([str(y) for y in sorted(x)])

dt.groupby(["Histology at Diagnosis"])["Time to Relapse (Days)*"].agg(['mean', 'median', 'count', get_origin_data])

,mean,median,count,get_origin_data
Histology at Diagnosis,,,,
Blastemal,182.000000,129.0,4,"94,120,138,376"
Epithelial tubular,80.000000,80.0,1,80
Mixed,311.571429,178.0,7,"68,91,107,178,186,246,1305"


##### 可以看到对于极低风险(肾母FH型-1期)中的胚芽型的平均复发天数为182天(中位时间129天), 混合型的平均是311天, 上皮型是80天

In [15]:
gene_11p15 = pd.read_csv("11p15.csv", sep="\t")
gene_11p15

,11p15 LOH,Relapse No,Relapse Yes,Total,P*
0,11p15 LOH,32 (80%),8 (20%),40,0.011
1,11p15 LOI,6 (75%),2 (25%),8,NaN
2,11p15 ROI,58 (96.7%),2 (3.3%),60,NaN
3,Total,96 (88.9%),12 (11.1%),108,NaN


####  可以看到肾母FH型-1期中携带11p15 LOH突变的患者有20%的概率会复发, p=0.011说明有统计学意义